In [1]:
import urllib3
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import plotly.express as px 
import numpy as np 

In [2]:
http = urllib3.PoolManager()
file_name = 'input.txt'
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
response = http.request("GET", url=url)
if response.status == 200:
    with open(file=file_name, mode='wb') as f:
        f.write(response.data)
        print('file download successfully.')
else:
    print(f'file not downloaded. check url!')

file download successfully.


In [3]:
with open(file=file_name, mode='r', encoding='utf-8') as f:
    text = f.read()

print(len(text))
print(text[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# create a mapping from characters to integers
stoi = { ch : i for i, ch in enumerate(chars) }
itos = { i : ch for i, ch in enumerate(chars) }

def encode(s):
    """takes a string, outputs list of integers"""
    return [stoi[ch] for ch in s]

def decode(l):
    """takes list of intergers, returns a string"""
    return ''.join([itos[i] for i in l])

print(encode('hii, there!'))
print(decode(encode('hii, there!')))

[46, 47, 47, 6, 1, 58, 46, 43, 56, 43, 2]
hii, there!


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text))
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n] 
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context.tolist()}, the target is {target}')

When input is [18], the target is 47
When input is [18, 47], the target is 56
When input is [18, 47, 56], the target is 57
When input is [18, 47, 56, 57], the target is 58
When input is [18, 47, 56, 57, 58], the target is 1
When input is [18, 47, 56, 57, 58, 1], the target is 15
When input is [18, 47, 56, 57, 58, 1, 15], the target is 47
When input is [18, 47, 56, 57, 58, 1, 15, 47], the target is 58


In [10]:
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    xb = torch.stack([data[i:block_size+i] for i in ix])
    yb = torch.stack([data[i+1:1+block_size+i] for i in ix])
    return xb, yb 

xb, yb = get_batch('train')
print(xb)
print(xb.shape)
print('---')

print(yb)
print(yb.shape)
print('---')

for b in range(batch_size): # batch dimension
    print(f'---Batch {b+1}---')
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'When input is {context}, the target is {target}')
    

tensor([[ 1, 39, 50, 51, 53, 57, 58,  1],
        [ 1, 51, 39, 52,  8,  0, 31, 47],
        [39, 58,  1, 44, 53, 56, 58, 46],
        [56, 57,  1, 39, 52, 42,  0, 32]])
torch.Size([4, 8])
---
tensor([[39, 50, 51, 53, 57, 58,  1, 58],
        [51, 39, 52,  8,  0, 31, 47, 56],
        [58,  1, 44, 53, 56, 58, 46, 61],
        [57,  1, 39, 52, 42,  0, 32, 53]])
torch.Size([4, 8])
---
---Batch 1---
When input is tensor([1]), the target is 39
When input is tensor([ 1, 39]), the target is 50
When input is tensor([ 1, 39, 50]), the target is 51
When input is tensor([ 1, 39, 50, 51]), the target is 53
When input is tensor([ 1, 39, 50, 51, 53]), the target is 57
When input is tensor([ 1, 39, 50, 51, 53, 57]), the target is 58
When input is tensor([ 1, 39, 50, 51, 53, 57, 58]), the target is 1
When input is tensor([ 1, 39, 50, 51, 53, 57, 58,  1]), the target is 58
---Batch 2---
When input is tensor([1]), the target is 51
When input is tensor([ 1, 51]), the target is 39
When input is tensor([ 1,

In [11]:
print(xb) #  our input to the transformer

tensor([[ 1, 39, 50, 51, 53, 57, 58,  1],
        [ 1, 51, 39, 52,  8,  0, 31, 47],
        [39, 58,  1, 44, 53, 56, 58, 46],
        [56, 57,  1, 39, 52, 42,  0, 32]])


In [16]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # become (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
logits, loss = model.forward(xb, yb)
print(logits.shape)
print(loss)
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

torch.Size([32, 65])
tensor(4.6015, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJt-wBpm&yiltNCjeO3:Cx&vvMYW-txjuAd IRFbTpJ$zkZelxZtTlHNzdXXUiQQY:qFINTOBNLI,&oTigq z.c:Cq,SDXzetn3XVjX-YBcHAUhk&PHdhcOb
nhJ?FJU?pRiOLQeUN!BxjPLiq-GJdUV'hsnla!murI!IM?SPNPq?VgC'R
pD3cLv-bxn-tL!upg
SZ!Uvdg CtxtT?hsiW:XxKIiPlagHIsr'zKSVxza?GlDWObPmRJgrIAcmspmZ&viCKot:u3qYXA:rZgv f:3Q-oiwUzqh'Z!I'zRS3SP rVchSFUIdd q?sPJpUdhMCK$VXXevXJFMl,i
YxA:gWId,EXR,iMC,$?srV$VztRwb?KpgUWFjR$zChOLm;JrDnDph
LBj,KZxJa


In [17]:
block_size = 3
batch_size = 2

xb_new, yb_new = get_batch('train')
print(xb_new)
print('---')
print(yb_new)

tensor([[54, 59, 58],
        [53, 60, 43]])
---
tensor([[59, 58,  1],
        [60, 43, 12]])


In [18]:
# "Formula" -> Maximum value in input < num embeddings. Why?
"""
Scenario A: It works
    - Layer: nn.Embedding(65, 65)
        - Vocabulary Size = 65 (Valid indices are 0 to 64)
    - Input: Shape (3, 2). Let's assume the data inside is [[12, 50], [2, 5]]
    - Result: All numbers (12, 50, 2, 5) are less than 65. The lookup succeeds.

Scenario B: It fails
    - Layer: nn.Embedding(10, 10)
        - Vocabulary Size = 10 (Valid indices are 0 to 9)
    - Input: Same input [[12, 50], [2, 5]]
    - Result: The model tries to look up index 12 and 50, but it only has rows 0-9 in its table. It chrashes.
"""
emb = nn.Embedding(61, 61)

In [19]:
emb = nn.Embedding(xb_new.max()+1, xb_new.max()+1)
logis = emb(xb_new)
logits.shape

torch.Size([32, 65])

In [20]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [35]:
batch_size = 32
for step in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.7681589126586914


In [36]:
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))




I acan tibs NESaieng whe toray MESI haifrall hat ut shoowhe, wowe kist gr maioog,
Y Yon the inchent ud thikehed pere'denl d T:
do's frifilles II mad cthy!
Fan t As who r wathe s thecorred tof pantedst ppry, e wite ivecoup INEN hed len tt gertay, lll ho m, a t g
A core f bl sthane heaig moy-kenkn RE f s iston My bere thamy EY waik porr'd.
Wis
AR:
INons we je ot'd g--gruenf ms; merainortl ru renafiu mat thricoistmu mangs fory
Sie ce LE ty I:
Fourcove, brd wen An allt tor wsit wishoure.
IN:
Prlyo


### The Mathematical trick in self-attention

In [75]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2   # batch, time, channels
x = torch.randn(B, T, C)
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [72]:
# version 1
# We want x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros(B, T, C)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [73]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, dim=1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) --> (B, T, C)

In [ ]:
# version 3: use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x 

False

In [45]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b 
print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [79]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 2, 3, 2   # batch, time, channels
x = torch.randn(B, T, C)
x

tensor([[[-2.0260, -2.0655],
         [-1.2054, -0.9122],
         [-1.2502,  0.8032]],

        [[-0.2071,  0.0544],
         [ 0.1378, -0.3889],
         [ 0.5133,  0.3319]]])

In [82]:
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [84]:
wei = torch.zeros((T, T))
wei

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [86]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])

In [89]:
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [91]:
x

tensor([[[-2.0260, -2.0655],
         [-1.2054, -0.9122],
         [-1.2502,  0.8032]],

        [[-0.2071,  0.0544],
         [ 0.1378, -0.3889],
         [ 0.5133,  0.3319]]])

In [92]:
wei @ x

tensor([[[-2.0260e+00, -2.0655e+00],
         [-1.6157e+00, -1.4889e+00],
         [-1.4939e+00, -7.2483e-01]],

        [[-2.0712e-01,  5.4416e-02],
         [-3.4641e-02, -1.6725e-01],
         [ 1.4801e-01, -8.5954e-04]]])

In [118]:
# version 4: self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)      # (B, T, 16)
q = query(x)    # (B, T, 16)

wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
# out = wei @ x # (B, T, T) @ (B, T, C) --> (B, T, C)

v = value(x)
out = wei @ v 
# out.shape 
out.shape

torch.Size([4, 8, 16])

In [111]:
out

tensor([[[ 1.8077e-01, -6.9988e-02, -3.5962e-01,  ..., -8.0164e-01,
           1.5236e+00,  2.5086e+00],
         [-5.3029e-01, -2.2274e-01,  7.9457e-01,  ...,  1.1658e+00,
           1.5962e+00,  5.5032e-02],
         [-5.9427e-01,  3.1861e-01,  5.8950e-02,  ..., -1.8886e-01,
          -4.4261e-02,  2.0092e-01],
         ...,
         [-1.6099e+00,  1.1770e+00,  1.1506e+00,  ..., -4.9985e-02,
           1.5179e+00, -5.6034e-01],
         [-7.3428e-01,  4.8980e-02,  5.2520e-01,  ...,  8.5586e-01,
           1.0967e+00,  3.4021e-01],
         [-7.5541e-01, -3.2941e-01,  5.4568e-01,  ..., -9.1224e-02,
          -1.4756e-01, -1.3229e-01]],

        [[ 4.5618e-01, -1.0917e+00, -8.2073e-01,  ...,  5.1187e-02,
          -6.5764e-01, -2.5729e+00],
         [ 9.4423e-02,  6.5213e-01, -1.1769e+00,  ...,  6.6192e-01,
          -1.0670e+00,  6.5763e-01],
         [ 1.6564e+00, -3.7253e-01,  1.1281e-01,  ..., -3.9861e-01,
           5.3721e-01, -1.1302e+00],
         ...,
         [ 3.3062e-01,  8

In [117]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [115]:
wei[0]

tensor([[-1.7629,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-3.3334, -1.6556,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-1.0226, -1.2606,  0.0762,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.7836, -0.8014, -0.3368, -0.8496,    -inf,    -inf,    -inf,    -inf],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,    -inf,    -inf,    -inf],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,    -inf,    -inf],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,    -inf],
        [-1.8044, -0.4126, -0.8306,  0.5899, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)

In [104]:
B, T, C = 4, 8, 16

In [95]:
a = torch.ones(B, T, C)
b = torch.ones(B, T, C)
a.shape, b.shape

(torch.Size([4, 8, 16]), torch.Size([4, 8, 16]))

In [ ]:
a @ b.transpose(-1, -2)

In [97]:
b.transpose(-1, -2).shape

torch.Size([4, 16, 8])

In [98]:
a.shape

torch.Size([4, 8, 16])

In [ ]:
# (4, 8, 16) @ (4, 16, 8) --> (4, 8, 8)

# (4, 8, 16)
# (4, 16, 8)
# |
# (4, 8, 8)